In [1]:
import pandas as pd
from func import cal_buoyancy,sounding_cal
import glob
import os
from haversine import haversine, Unit
import re
import datetime
import metpy.calc as mpcalc
from metpy.units import units
from metpy import constants
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
from scipy.interpolate import interp1d
from scipy.optimize import bisect
import numpy as np
import pickle
import numpy as np
import math
import pint
import seaborn as sns
from pydoc import help
from sklearn.preprocessing import QuantileTransformer
import xarray as xr
import netCDF4 as nc
import re
import matplotlib.pyplot as plt
from numpy import load
from numpy import asarray
from numpy import save
import pytz
from scipy import stats
from sklearn.linear_model import LinearRegression
from collections import Counter
import pymannkendall as mk
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
with open('soundings_fig1&4.pkl', 'rb') as file:
    soundings = pickle.load(file)
soundings.shape

(2626, 11)

In [5]:
#gaoyoun's method

def cal_ctp_hi(temperature,dewpoint,pressure,altitude):
	'''
	INPUT:
		temperature 
		dewpoint
		pressure
		altitude 
  Note: all input should have metpy.units but altitude
	OUTPUT:
		ctp  : Convectie trigering Potential
		hi	  : Humidity index
		moist_t : moist adiabatic temperature profile from 1km to 3km AGL
		cal_p   : Pressure profile from 1km to 3km AGL
	'''
	alt = altitude    
	start_index = np.where(alt == alt[0]+1000)[0][0]
	end_index   = np.where(alt == alt[0]+3000)[0][0]
	cal_p = pressure[start_index:end_index+1]
	cal_t = temperature[start_index:end_index+1]

	moist_t = mpcalc.moist_lapse(pressure = cal_p, 
	            temperature = cal_t[0])
	p_level = np.arange(1000,500,-2)*units('hPa')
	moist_temp = mpcalc.moist_lapse(pressure = p_level, 
	            temperature = cal_t[0],
	            reference_pressure = cal_p[0]).to(units.degC)
	moist_t = np.interp(cal_p.magnitude[::-1],p_level.magnitude[::-1],moist_temp.magnitude[::-1])[::-1]*moist_temp.units
	#ctp0 = np.trapz((moist_t-cal_t)[::-1],cal_p[::-1])
	ctp1 = (constants.Rd*(np.trapz((moist_t.magnitude-cal_t.magnitude)[::-1],np.log(cal_p.magnitude)[::-1])* units.degK)).to(units('J/kg'))    

	index0 = np.where(alt == alt[0]+500)[0][0]
	index1 = np.where(alt == alt[0]+1500)[0][0]
	hi  = temperature[index0]-dewpoint[index0]+temperature[index1]-dewpoint[index1]
	return ctp1.magnitude,hi.magnitude,moist_t.magnitude,cal_p

In [6]:
soundings['ctp']=""

soundings['hilow']=""

for i in range(soundings.shape[0]):
    
    row = soundings.iloc[i,:]

    temperature = row['interp_temp'] * units.degC

    dewpoint = row['interp_dp']  * units.degC

    pressure = row['interp_pres'] * units.hPa

    altitude = row['interp_alt']

    ctphi = cal_ctp_hi(temperature, dewpoint, pressure, altitude)
    
    soundings['ctp'][i] = ctphi[0]
    
    soundings['hilow'][i] = ctphi[1]


In [8]:
#for figure 2&3 reproduction

with open('soundings_ctp_hilow.pkl', 'wb') as file:
    
    pickle.dump(soundings, file)

In [9]:
soundings.shape

(2626, 13)

Running cal_buoyancy in order to calculate CAPE

In [7]:
# %run /home/annierosen16/func.ipynb

#calculate cape test, too large with this method

# soundings['cape']=""

# for i in range(soundings.shape[0]):

#     row = soundings.iloc[i,:]

#     t = row['interp_temp'] * units.degC

#     dp = row['interp_dp']  * units.degC

#     p = row['interp_pres'] * units.hPa

#     pres, temp, dewpt, prof = mpcalc.parcel_profile_with_lcl(p, t, dp)

#     cape, cin = cape_cin(pressure=pres, temperature=temp, dewpoint=dewpt, parcel_profile=prof)
    
#     soundings['cape'][i] = cape.magnitude
    

# cape_mean = soundings.groupby('tercile')['cape'].mean()
    
# print(cape_mean)

In [8]:
#cape test with gaoyun code

# %run /home/annierosen16/func2.ipynb

# %run /home/annierosen16/func.ipynb

# soundings['capetest']=""

# for i in range(soundings.shape[0]):

#     row = soundings.iloc[i,:]

#     t = row['interp_temp'] * units.degC

#     dp = row['interp_dp']  * units.degC

#     p = row['interp_pres'] * units.hPa

#     h = row['interp_alt'] * units.meter

#     capetest = sounding_cal(pressure=p, temperature=t, dewpt=dp)
#     print(capetest.p_buoyancy.magnitude)
    
#     soundings['capetest'][i] = capetest.p_buoyancy.magnitude
    

# result = sounding_cal_byH(height=h, press=p, temperature=t , dewpt=dp, parcel_temperature_profile=None,specific_height_start=None, specific_height_end=None, low_alt = [0,])

# cape_meantest = soundings.groupby('tercile')['capetest'].mean()

# prof = parcel_profile_with_lcl(p, t, dp)

# plt.plot(prof[3],prof[0])
# plt.gca().invert_yaxis()

In [23]:
# annie -- hilow

# def calculate_hi_low(row):
    
#     pres = row['interp_pres']

#     _50_hpa = pres[0] - 50
#     _50_hpa_index = np.argmax(pres <= _50_hpa)

#     _150_hpa = pres[0] - 150
#     _150_hpa_index = np.argmax(pres <= _150_hpa)

#     tdry_50 = row['interp_temp'][_50_hpa_index]
#     dp_50 = row['interp_dp'][_50_hpa_index]

#     tdry_150 = row['interp_temp'][_150_hpa_index]
#     dp_150 = row['interp_dp'][_150_hpa_index]

#     return (tdry_150 - dp_150) + (tdry_50 - dp_50)

# soundings['hilow'] = soundings.apply(calculate_hi_low, axis=1)

# soundings.shape

(2435, 16)

In [24]:
annie -- ctp

def calculate_ctp(row):
    
    alt = row['interp_alt']

    pres = row['interp_pres'] * units.hPa

    altmin = 1000

    altmax = 3000

    index_1000m_agl = np.argwhere(alt >= altmin).min()

    index_3000m_agl = np.argwhere(alt >= altmax).min()

    pressure_at_1000m_agl = pres[index_1000m_agl]

    alt_range = (alt >= altmin) & (alt <= altmax)

    tdry = row['interp_temp'] * units.degC

    temp_at_1000m_agl = tdry[index_1000m_agl]

    moist_lapse_profile = mpcalc.moist_lapse(pres, temp_at_1000m_agl, reference_pressure=pressure_at_1000m_agl)

    tparcel = np.array(moist_lapse_profile[alt_range])
    
    tparcel = tparcel + 273.15

    tenv = np.array(tdry[alt_range])
    
    tenv = tenv + 273.15

    y = (tparcel - tenv)/tenv

    x = alt[alt_range]
    
    presrange = pres[alt_range]

#     ctp = constants.g*np.trapz(y, x)
    
#     gaoyun's method:
    ctp = (constants.Rd*(np.trapz((tparcel-tenv)[::-1],np.log(presrange.magnitude)[::-1])* units.degK)).to(units('J/kg')) 
    
#     print(abs(ctp.magnitude-ctpg.magnitude))
    
    return ctp.magnitude
    

soundings['ctp'] = soundings.apply(calculate_ctp, axis=1)

plt.figure(figsize=(6, 4))
plt.hist(soundings['ctp'], bins=30, edgecolor='black')
plt.xlabel('ctp')
plt.title('CTP Distribution')
plt.ylabel('Frequency')
plt.show()

In [43]:
# comp = soundings[['ctp','ctp_annie','hilow','hilow_annie']]